In [17]:
import cohere
import pandas as pd
# Instantiate the Cohere client
api_key="0DlpJISOe7M8igl3vZPGJNPeTYs7hgNIbS7uhl5A"
co = cohere.Client(api_key)

# Load your CSV file
file_path = "ACLED 2023 Subset.csv"  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

# Select five random examples of each event type
sampled_df = df.groupby('event_type').apply(lambda x: x.sample(min(len(x), 10))).reset_index(drop=True)

# Keep only the 'event_type' and 'notes' columns
sampled_df = sampled_df[['event_type', 'notes']]

# Rename columns: 'event_type' to 'label' and 'notes' to 'text'
sampled_df.rename(columns={'event_type': 'label', 'notes': 'text'}, inplace=True)

# Save the result in CSV format for Cohere
sampled_df.to_csv("cohere_prepared_single_label.csv", index=False)

# Optionally, save in JSONL format
sampled_df.to_json("cohere_prepared_single_label.jsonl", orient="records", lines=True)

# Display the first few rows to verify
print(sampled_df)


                         label  \
0                      Battles   
1                      Battles   
2                      Battles   
3                      Battles   
4                      Battles   
5                      Battles   
6                      Battles   
7                      Battles   
8                      Battles   
9                      Battles   
10  Explosions/Remote violence   
11  Explosions/Remote violence   
12  Explosions/Remote violence   
13  Explosions/Remote violence   
14  Explosions/Remote violence   
15  Explosions/Remote violence   
16  Explosions/Remote violence   
17  Explosions/Remote violence   
18  Explosions/Remote violence   
19  Explosions/Remote violence   
20                    Protests   
21                    Protests   
22                    Protests   
23                    Protests   
24                    Protests   
25                    Protests   
26                    Protests   
27                    Protests   
28            

C:\Users\gamal\AppData\Local\Temp\ipykernel_26828\2726814160.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('event_type').apply(lambda x: x.sample(min(len(x), 10))).reset_index(drop=True)


In [21]:
import cohere
from cohere.finetuning import BaseModel, FinetunedModel, Settings
from cohere.datasets import ParseInfo
from dotenv import load_dotenv

import os

load_dotenv()


# Instantiate the Cohere client
api_key=os.getenv("COHERE_API_KEY")
if not api_key:
    raise ValueError("API key not found! Please set the COHERE_API_KEY in your .env file.")
co = cohere.Client(api_key)

# Step 1: Create Dataset for Single-Label Fine-Tuning
single_label_dataset = co.datasets.create(
    name="single-label-dataset",
    data=open("cohere_prepared_single_label.csv", "rb"),  # Provide the correct path to your dataset
    type="single-label-finetune-input",
    parse_info=ParseInfo(delimiter=",")
)

# Wait for the dataset to be validated
print(single_label_dataset.await_validation())

# Step 2: Start the Fine-Tuning Job
finetune = co.finetuning.create_finetuned_model(
    request=FinetunedModel(
        name="single-label-ft",
        settings=Settings(
            base_model=BaseModel(base_type="BASE_TYPE_CLASSIFICATION"),
            dataset_id=single_label_dataset.id,
        ),
    ),
)

# Wait for fine-tuning to complete and print the status
print(f"Fine-tune ID: {finetune.id}, Fine-tune Status: {finetune.status}")

# Step 3: Use the Fine-Tuned Model for Classification
ft = co.finetuning.get_finetuned_model(finetune.id)

response = co.classify(
    inputs=["Classify this example text!"],
    model=ft.id + "-ft",  # Append "-ft" to the model ID as per Cohere's requirement
)

# Print the model's classification response
print(response.classifications)


BadRequestError: status_code: 400, body: {'message': 'invalid dataset type'}